# 1. Import Libraries

In [ ]:
import math # Used for algebraic calculation only (tan angle on edges)
import matplotlib.pyplot as plt # Used to display the image as plot of points only
from PIL import Image # Used to obtain the image only

# 2. Edge Detection

## 2.1 Dimensions of the Object

In [ ]:
def getDimensions(image):
    width, height = image.size
    return width, height

## 2.2 Convert RGB Color to Grayscale Color

In [ ]:
def rgbToGreyscale(image):
    width, height = getDimensions(image)
    grey_img = Image.new("L", (width, height))
    pixels = image.load()
    for x in range(width):
        for y in range(height):
            r, g, b = pixels[x, y]
            grey = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            grey_img.putpixel((x, y), grey)
    return grey_img

## 2.3 Compute the First Derivative

In [ ]:
def firstDerivative(img):
    width, height = getDimensions(img)
    fx = Image.new("L", (width, height))
    fy = Image.new("L", (width, height))

    img_pixels = img.load()
    fx_pixels = fx.load()
    fy_pixels = fy.load()

    for x in range(1, width - 1):
        for y in range(1, height - 1):
            hx = 0.1  # Adjust hx to a smaller value
            hy = 0.1 # Adjust hy to a smaller value
            fx_val = int((img_pixels[x + 1, y] - img_pixels[x - 1, y]) / (2 * hx))  # Use central differences in x direction
            fy_val = int((img_pixels[x, y + 1] - img_pixels[x, y - 1]) / (2 * hy))  # Use central differences in y direction
            fx_pixels[x, y] = fx_val
            fy_pixels[x, y] = fy_val
    return fx, fy

## 2.4 Compute the Second Derivative

In [ ]:
#Note that, we did not use second derivative since it is too sensitive to the noise in our case
def secondDerivatives(img):
    width, height = getDimensions(img)
    fxx = Image.new("L", (width, height))
    fyy = Image.new("L", (width, height))

    img_pixels = img.load()
    fxx_pixels = fxx.load()
    fyy_pixels = fyy.load()

    for x in range(1, width - 1):
        for y in range(1, height - 1):
            hx = 0.8  # Adjust hx to a smaller value
            hy = 0.8  # Adjust hy to a smaller value
            fxx_val = int(
                (img_pixels[x + 1, y] - 2 * img_pixels[x, y] + img_pixels[x - 1, y]) / (hx ** 2)
            )  # Use central differences in x direction
            fyy_val = int(
                (img_pixels[x, y + 1] - 2 * img_pixels[x, y] + img_pixels[x, y - 1]) / (hy ** 2)
            )  # Use central differences in y direction
            fxx_pixels[x, y] = fxx_val
            fyy_pixels[x, y] = fyy_val
    return fxx, fyy

## 2.5 Compute the Combine Edges

In [ ]:
# We combine the edge data of X axis and Y axis
def combineEdges(fx, fy):
    width, height = getDimensions(fx)
    combined_img = Image.new("L", (width, height))

    fx_pixels = fx.load()
    fy_pixels = fy.load()
    combined_pixels = combined_img.load()

    for x in range(width):
        for y in range(height):
            edge_x = fx_pixels[x, y]
            edge_y = fy_pixels[x, y]
            combined_edge = int((edge_x**2 + edge_y**2)**0.5)  # Combine edges using Euclidean distance
            combined_pixels[x, y] = combined_edge
    return combined_img

## 2.6 Convert Grayscale Edge Data to Array

In [ ]:
def grayscale_to_array(image_path):
    width, height = image_path.size
    pixel_data = list(image_path.getdata())  # Get pixel values as a flat list

    # Reshape the flat list into a 2D array
    array = [pixel_data[i:i+width] for i in range(0, len(pixel_data), width)]
    return array

## 2.7 Thresholding & Denoising Edge

In [ ]:
def thresholding(array, threshold):
    for i in range (len(array)):
        for j in range (len(array[0])):
            if array[i][j]>threshold:
                array[i][j]=255
    return array

def reduceNoise(array):
    for i in range(1, len(array)):
        for j in range(1, len(array[0])):
            if array[i][j]==255 and ((array[i-1][j]==255 and array[i][j-1]==255 and array[i+1][j]==255)
                                     or (array[i-1][j]==255 and array[i][j-1]==255 and array[i][j+1]==255)
                                     or (array[i-1][j]==255 and array[i+1][j]==255 and array[i][j+1]==255)
                                     or (array[i+1][j]==255 and array[i][j+1]==255 and array[i][j-1]==255)):
                                     array[i][j] = 0
    return array

## 2.8 Get the array point into List


In [ ]:
def getPoints(array):
    lis = []
    for j in range (len(array[0])):
        for i in range (len(array)):
            if array[i][j]==255:
                lis.append((i, j))
    return lis


def sortPoints(points):
    centroid_x = sum(point[0] for point in points) / len(points)
    centroid_y = sum(point[1] for point in points) / len(points)
    centroid = (centroid_x, centroid_y)

    for i, point in enumerate(points):
        delta_x = point[0] - centroid_x
        delta_y = point[1] - centroid_y
        if delta_x == 0:
            angle = 0 if delta_y >= 0 else math.pi
        else:
            angle = math.atan(delta_y / delta_x)
            if delta_x < 0:
                angle += math.pi
                points[i] = (point[0], point[1], angle)
                points.sort(key=lambda p: p[2])

                corners = []
                threshold = 0 # Adjust the threshold
                for i in range(len(points)):
                    prev_angle = points[i-1][2] if i > 0 else points[-1][2]
                    curr_angle = points[i][2]
                    next_angle = points[(i+1) % len(points)][2]
                    angle_diff = abs(curr_angle - prev_angle) + abs(curr_angle - next_angle)
                    if angle_diff > threshold:
                        corners.append((points[i][0], points[i][1]))
                return corners

# 3. Interpolation Steps

## 3.1 Lagrange Method

In [ ]:
def lagrange_interpolation(x, y, x_interpolated):
    n = len(x)
    y_interpolated = []
    for xi in x_interpolated:
        interpolated_value = 0
        for j in range(n):
            basis = 1
            for k in range(n):
                if k != j and x[j] != x[k]:
                    basis *= (xi - x[k]) / (x[j] - x[k])
            interpolated_value += y[j] * basis
        y_interpolated.append(interpolated_value)
    return y_interpolated

## 3.2 Barycentric Method

In [ ]:
def barycentric_interpolation(x, y, x_interpolated):
    n = len(x)
    weights = []

    for i in range(n):
        weight = 1
        for j in range(n):
            if i != j and x[i]!=x[j]:
                weight /= (x[i] - x[j])
        weights.append(weight)
    y_interpolated = []
    for xi in x_interpolated:
        numerator_sum = 0
        denominator_sum = 0
        for i in range(n):
            if xi == x[i]:
                interpolated_value = y[i]
                break
            weight = weights[i] / (xi - x[i])
            numerator_sum += weight * y[i]
            denominator_sum += weight
        else:
            interpolated_value = numerator_sum / denominator_sum
        y_interpolated.append(interpolated_value)
    return y_interpolated

## 3.3 Standard Method

In [ ]:
def interpolate_points(edges, num_points):
    interpolated_points = []
    for i in range (1, len(edges)-1):
        start_point = edges[i]
        end_point = edges[i+1]

        # Interpolate the points along the edge
        step_x = (end_point[0] - start_point[0]) / (num_points + 1)
        step_y = (end_point[1] - start_point[1]) / (num_points + 1)

        for i in range(num_points):
            x = start_point[0] + (step_x * (i + 1))
            y = start_point[1] + (step_y * (i + 1))
            interpolated_points.append([x, y])

    return interpolated_points

## 3.4 Applying the Interpolation Methods to the Edges

In [ ]:
def apply_interpolation(edges, method):
    x_coords, y_coords = zip(*edges)
    x_interpolated = [xi for xi in range(int(min(x_coords)), int(max(x_coords))+1)]

    if method == 'Lagrange':
        y_interpolated = lagrange_interpolation(x_coords, y_coords, x_interpolated)
    elif method == 'Barycentric':
        y_interpolated = barycentric_interpolation(x_coords, y_coords, x_interpolated)
    elif method == 'Standard':
        interpolated_points = interpolate_points(edges, 15)
        return interpolated_points


# 4. Volume Estimation

## 4.1 Simpson Method

In [ ]:
def f(x):
    return 1

def simpson_integration(a, b, n):
    h = (b - a) / n
    x = a
    sum1 = 0
    sum2 = 0

    for i in range(1, n, 2):
        x += h
        sum1 += f(x)
        x = a + h
        for i in range(2, n, 2):
            x += h
            sum2 += f(x)
        integral = h / 3 * (f(a) + f(b) + 4 * sum1 + 2 * sum2)
        return integral

def compute_volume_simpson(edges, n):
    volume = 0

    for i in range(len(edges) - 1):
        x1, y1 = edges[i]
        x2, y2 = edges[i + 1]
        area = (y1 + y2) / 2 * (x2 - x1)
        integral = simpson_integration(x1, x2, n)
        volume += area * integral
    return volume

## 4.2 Voxel Grid Method

In [ ]:
def compute_volume_voxelpt2(edges):
    # Determine the minimum and maximum coordinates of the image
    min_y = min(edges, key=lambda edge: edge[0])[0]
    max_y = max(edges, key=lambda edge: edge[0])[0]
    min_x = min(edges, key=lambda edge: edge[1])[1]
    max_x = max(edges, key=lambda edge: edge[1])[1]

    # Initialize the volume, voxel size, and tuning
    tuning = 3750
    volume = 0
    voxel_size = 2  # Assume that each voxel represents a unit square

    # Iterate over each voxel and check if it intersects with the image edges
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            # Check if the voxel intersects with any edge
            intersect = False
            for i in range(len(edges)):
                j = (i + 1) % len(edges)
                if (edges[i][1] > y) != (edges[j][1] > y) and x < (
                        edges[j][0] - edges[i][0]) * (y - edges[i][1]) / (
                        edges[j][1] - edges[i][1]) + edges[i][0]:
                    intersect = not intersect

            # If the voxel intersects with an odd number of edges, it is part of the image
            if intersect:
                volume += voxel_size * tuning

    return volume

# 5. Implementation

In [ ]:
image = Image.open('box.jpg')
grey_img = rgbToGreyscale(image)
fx, fy = firstDerivative(grey_img)
combined_edges = combineEdges(fx, fy)
array = grayscale_to_array(combined_edges)
array = thresholding(array, 255)
array = reduceNoise(array)
points = getPoints(array)

# Object Volume Detection with Simpson Method
volume = compute_volume_simpson(points, 30)
print("Raw Volume of an Object using Simpson Method:", volume)
volume_estimation = volume / (28.346 ** 3)
print("Volume of an Object after Scaling:", volume_estimation)
error = (volume_estimation - 5175) / 5175
print("Error:", error * 100)

print("----------------------------------------------------------------")

# Object Volume Detection with Voxel Grid Method
volume3 = compute_volume_voxelpt2(points)
print("Raw Volume of an Object using Voxel Grid:", volume3)
volume3_estimation = volume3 / (28.346 ** 3)
print("Volume of an Object after Scaling:", volume3_estimation)
error = (volume3_estimation - 5175) / 5175
print("Error:", error * 100)

Raw Volume of an Object using Simpson Method: 343392417.0555565
Volume of an Object after Scaling: 15077.014492209133
Error: 191.34327521177067
----------------------------------------------------------------
Raw Volume of an Object using Voxel Grid: 127852500
Volume of an Object after Scaling: 5613.501928475322
Error: 8.473467216914427


In [ ]:
len(points)

1355

In [ ]:
im = plt.imshow(array, cmap='Greys')
plt.show()